# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.dataset import Dataset
import joblib
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
import json
import numpy as np
from azureml.automl.core.shared import constants
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Workspace and Experiment
ws = Workspace.from_config()

experiment_name = 'automl_experiment'
experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Creating Compute Cluster
cluster_name = "automl-cluster"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_V2', 
        max_nodes=4
    )

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
# Adding dataset from workspace
dataset = Dataset.get_by_name(ws, name='Heart_Failure_Clinical_Records_Dataset')
Heart_Failure_Clinical_Records_Dataset = dataset.to_pandas_dataframe()

Heart_Failure_Clinical_Records_Dataset.head()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# AutoML settings in JSON format
automl_settings = {
    "experiment_timeout_minutes":25,
    "task":'classification',
    "primary_metric":'accuracy',
    "training_data":dataset,
    "label_column_name":'DEATH_EVENT',
    "n_cross_validations":4,
    "max_concurrent_iterations": 4,
    "featurization": 'auto'
}

# AutoML config passing the settings and the compute resource
automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    **automl_settings
)

In [ ]:
# Submit the experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# Showing the output might not work with the newest version of AzureML
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Printing details of the best run
print('Accuracy: ', remote_run.get_metrics('accuracy'))

best_automl_run, best_automl_model = automl_run.get_output()
print('\n Best run: ', best_automl_run)
print('\n Model: ', best_automl_model)
print('\n Best run details: ', best_automl_run.get_details())
print('\n Final estimator: ', best_automl_model._final_estimator)

In [ ]:
# Save your best automl model
# when not working save it as joblib file
joblib.dump(best_automl_run, 'best-automl-run.pkl')
joblib.dump(best_automl_model, 'best-automl-model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Can be done in Azure ML Workspace but here within the notebook
model = Model.register(
    workspace=ws,
    model_name = "best-automl-model", 
    model_path = './best-automl-model.pkl')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
print(model.name, model.id, model.version, sep='\t')

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(
    name='sklearn-env', 
    file_path='conda_dependencies.yml'
)
inference_config = InferenceConfig(
    entry_script='score_automl.py', 
    environment=env
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    auth_enabled=True,
    tags = {'name':'failure Prediction'},
    description='Heart failure prediction model'
    )

service = model.deploy(
    workspace=ws,
    name="automl-deploy",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
    )

service.wait_for_deployment(show_output=True)

In [ ]:
# Enable Logging and check state
service.update(enable_app_insights=True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# URL for the web service
######## ToDo:
scoring_uri = ''
# If the service is authenticated, set the key or token
######## ToDo:
key = ''

data = {
  "data": [
            "Inputs":{
                "data": [
                    {
                        'age': 75.0, 
                        'anaemia': 0.0, 
                        'creatinine_phosphokinase': 582.0,
                        'diabetes': 0.0,
                        'ejection_fraction': 20.0,
                        'high_blood_pressure': 1.0,
                        'platelets': 265000.0,
                        'serum_creatinine': 1.9,
                        'serum_sodium': 130.0,
                        'sex': 1.0,
                        'smoking': 0.0,
                        'time': 4.0          
                    }
                ]   
            },
            "GlobalParameters": {
            "method": "predict"
            }
        ]

}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(
    scoring_uri, 
    input_data, 
    headers=headers
    )
    
print("prediction ", resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#print the logs of the web service
print(service.get_logs())

#delete the service
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
